<a href="https://colab.research.google.com/github/atstuyuki/ScikitLearn/blob/main/yolov8_predict_ON_20240312.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Mount your Google Drive Folder


In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd ./drive/MyDrive

Mounted at /content/drive
/content/drive/MyDrive


# Install libraries

In [2]:
%pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 721.2/721.2 kB 5.0 MB/s eta 0:00:00


# Upload your pretrained .pt file
# 訓練済みのbest.ptファイルをuploadする
## best.ptはwightsフォルダに入ってます
## using yolov8x model will takes a while
## otherwise put your best.pt model in Google Drive Folder and copy the path using right click

In [11]:
# prompt: GUIでbest.ptファイルをuploadしてpt_modelという変数に格納

from google.colab import files

uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  pt_model = fn


Saving best.pt to best.pt
User uploaded file "best.pt" with length 136733609 bytes


# Upload your Image data
## repeat this cell and below when analyzing several images

In [4]:
# prompt: GUIで画像データをuploadしてimageという変数に格納

from google.colab import files

# Upload images using the file upload widget
uploaded = files.upload()

# Store the uploaded images in a variable called "image"
image = list(uploaded.keys())

# Print the names of the uploaded images
print(image)


Saving Test data (20).jpg to Test data (20).jpg
['Test data (20).jpg']


# Predict your Image with YOLOv8
##'runs'folder will be created in your GoogleDrive Folder
## the result will be saved in 'runs/detect/predict' folder

In [12]:
from ultralytics import YOLO
#used the uploded model
model = YOLO(pt_model)
#otherwise copy the path of your model and paste between ""
#model = YOLO("/content/drive/MyDrive/osteonecrosis_shinohara_3classes/yolov8x_20230312/weights/best.pt")
predicted = model.predict(image,show=True, save=True,save_txt=True, save_conf=True, imgsz=320, conf=0.25, show_labels=False,show_conf=False)

WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()


0: 320x320 8 Osteocytess, 11 Pyknotic osteocytess, 38 Empty lacunaes, 38.7ms
Speed: 1.4ms preprocess, 38.7ms inference, 3.4ms postprocess per image at shape (1, 3, 320, 320)
Results saved to runs/detect/predict6
1 label saved to runs/detect/predict6/labels


In [13]:
# count each labels

for result in predicted:
    boxes = result.boxes  # Boxes object for bounding box outputs
boxes_cls_list = boxes.cls.tolist()
boxes_cls_list = [int(x) for x in boxes_cls_list]
boxes_cls_list.sort()
# prompt: boxes_cls_listの各要素を数えてprintする、さらにboxes_cls_listの長さで割り100をかけて%表記でprintする
count_dict = {}
print(model.names)
for cls in boxes_cls_list:
  if cls not in count_dict:
    count_dict[cls] = 0
  count_dict[cls] += 1

for cls, count in count_dict.items():
  print(f"Class {cls}: {count}")
  print(f"Percentage: {(count / len(boxes_cls_list)) * 100:.2f}%")

{0: 'Osteocytes', 1: 'Pyknotic osteocytes', 2: 'Empty lacunae'}
Class 0: 8
Percentage: 14.04%
Class 1: 11
Percentage: 19.30%
Class 2: 38
Percentage: 66.67%


# Save count data as CSV in Google Drive Folder

In [14]:
# prompt: 各クラスの数とPersentageを表形式のデータにして保存
# CSVの名前にはimageの名前を反映

import csv

# Create a CSV file with the image name as the filename
csv_file_name = f"{image[0]}.csv"

# Write the header row
with open(csv_file_name, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['Class', 'Count', 'Percentage'])

    # Write the data rows
    for cls, count in count_dict.items():
        percentage = (count / len(boxes_cls_list)) * 100
        writer.writerow([model.names[cls], count, f'{percentage:.2f}%'])

print(f'Saved class count data in GoogleDrive Folder as: {csv_file_name}')


Saved class count data in GoogleDrive Folder as: Test data (20).jpg.csv
